<a href="https://colab.research.google.com/github/shlee0619/workStats/blob/main/0108.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import argparse

In [ ]:
# 템플릿 매칭

# 입력영상
img = cv2.imread('./data/circuit.bmp', cv2.IMREAD_COLOR)
# 템플릿영상
templ = cv2.imread('./data/crystal.bmp', cv2.IMREAD_COLOR)
# 입력영상 밝기 증가
img = img + (50, 50, 50)
# 표준편차 10인 가우시안 잡음 추가
noise = np.zeros(img.shape, np.int32)
cv2.randn(noise, 0, 10)
img = cv2.add(img, noise, dtype=cv2.CV_8UC3)
# Normalization 적용 후 영상 저장
res = cv2.matchTemplate(img, templ, cv2.TM_CCOEFF_NORMED)
# res = cv2.matchTemplate(img, templ, cv2.TM_SQDIFF)
res_norm = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
# 결과 행렬에서 최댓값 위치 저장 후 확인
print(cv2.minMaxLoc(res))
minv, maxv, minloc, maxloc = cv2.minMaxLoc(res)
print('maxv:', maxv)
# print('minv:', minv)

(th, tw) = templ.shape[:2]
cv2.rectangle(img, maxloc, (maxloc[0] + tw, maxloc[1] + th), (0, 0, 255), 3)

cv2.imshow('Original', img)
cv2.imshow('Template', templ)
# cv2.imshow('res_norm', res_norm)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 연결요소 찾기 = 레이블링
# 입력 영상
src = np.array([[0, 0, 1, 1, 0, 0, 0, 0],
                [1, 1, 1, 1, 0, 0, 1, 0],
                [1, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 1, 0],
                [0, 0, 0, 1, 1, 1, 1, 0],
                [0, 0, 0, 1, 0, 0, 1, 0],
                [0, 0, 1, 1, 1, 1, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0]]).astype(np.uint8)

src = src * 255

cnt, labels = cv2.connectedComponents(src)

print('src:'),
print(src)
print('labels:')
print(labels)
print('number of labels:', cnt)

In [ ]:
# 레이블링 응용
src = cv2.imread('./data/keyboard.bmp', cv2.IMREAD_GRAYSCALE)
# 오츠 이진화
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# 각 객체 영역의 통계 정보
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)
# 3채널 영상 변환
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
# 흰색 객체 영역 반복
for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]
    # 픽셀 개수가 20보다 작으면 무시
    if area < 20:
        continue
    # 검출된 객체에 사각형 그리기
    pt1 = (x, y)
    pt2 = (x + w, y + h)
    cv2.rectangle(dst, pt1, pt2, (0, 0, 255))

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()